fastText indilir ve oluşturulur


In [0]:
!git clone https://github.com/facebookresearch/fastText.git

In [0]:
!pip install ./fastText/

Classification'da kullanılacak olan verisetimizi alıyoruz.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
PREPROCESSED_DATASET_WITH_STEMMER = "gdrive/My Drive/mbti/preprocessed_dataset_with_stemming.csv"
PREPROCESSED_DATASET_WITHOUT_STEMMER = "gdrive/My Drive/mbti/preprocessed_dataset_no_stemming.csv"
PREPROCESSED_DATASET_ZEMBEREK = "gdrive/My Drive/mbti/preprocessed_dataset_zemberek"
TRIMMED_DATASET = "gdrive/My Drive/mbti/trimmed_dataset.csv"
RAW_DATASET = "gdrive/My Drive/mbti/all_users_v2.csv"

In [0]:
import pandas as pd 
df = pd.read_csv(PREPROCESSED_DATASET_WITHOUT_STEMMER, sep = ';', header = 0)

In [0]:
df

Olmayan değerler silinir

In [0]:
df = df.dropna().reset_index(drop = True)

In [0]:
df

fastText classification yaparken labellı verinin ``` __label__LabelValue ``` şeklinde olmasını bekliyor. Burada ilgili dataframe'i bu hale getirilir

In [0]:
df['typeClass'] = ["__label__" + class_label for class_label in df['typeClass']]

df

Veriseti %80 train %20 test olacak şekilde ikiye ayrılır. Train veriseti ile ilgili model oluşturulacaktır.

In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, shuffle = True, random_state = 42)


In [0]:
train

In [0]:
test

Train ve test olan verisetlerimizin bir dosyaya kaydedilip ilgili pathlerinin fastText'e geçilmesi gerekiyor. O yüzden bu adımda ilgili verisetleri dosyalara kaydedilir. quoting = csv.QUOTE_NONE kaydedilen string değerlerin herhangi bir quote (",') karakteriyle sarılmamasını söyler

In [0]:
import csv

In [0]:
TRAIN_DATASET_PATH = "gdrive/My Drive/mbti/fastText_train_set.txt"
train.to_csv(TRAIN_DATASET_PATH, header = False, index = False, columns = ['typeClass', 'entry'], sep = ' ', quoting = csv.QUOTE_NONE, escapechar = ' ')

In [0]:
TEST_DATASET_PATH = "gdrive/My Drive/mbti/fastText_test_set.txt"
test.to_csv(TEST_DATASET_PATH, header = False, index = False, columns = ['typeClass', 'entry'], sep = ' ', quoting = csv.QUOTE_NONE, escapechar=' ')

FastText'i kullanılabilmek için ilgili kütüphaneyi import ediyoruz.

In [0]:
import fasttext

Modelimizi varsayılan parametrelerle eğitiyoruz

In [0]:
mbti_model = fasttext.train_supervised(TRAIN_DATASET_PATH)

Oluşturulan modeli kaydediyoruz.

In [0]:
mbti_model.save_model('gdrive/My Drive/mbti/fastText_typeClass_model.bin')

Sonuçları tutmak için gerekli olan dictionary oluşturulur.

In [0]:
results = {
    'predicted': {
        'analysts': {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}},
        'diplomats': {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}},
        'explorers': {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}},
        'sentinels': {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
    }

}

In [0]:
test = test.reset_index(drop = True)

In [0]:
test

In [0]:
for i in range(test.shape[0]):
  y_actual = str(test['typeClass'][i]).replace('__label__', '')

  test_entry = test['entry'][i]
  y_predicted = mbti_model.predict(test_entry, k = 1)[0][0]
  y_predicted = y_predicted.replace('__label__', '')

  results['predicted'][y_predicted]['actual'][y_actual] += 1

In [0]:
results